In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
df = pd.read_csv('/kaggle/input/nonoiseoutliersrms/development_noNoise_noRMS_noOutliers.csv')

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:

def euclidean_distance_loss(y_true, y_pred):
    squared_difference = np.square(y_true - y_pred)
    summation = np.sum(squared_difference, axis=0)
    sqrt_diff = np.sqrt(summation)
    summation2 = np.sum(sqrt_diff, axis=0)
    return summation2/squared_difference.shape[1]


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.multioutput import MultiOutputRegressor

# Sample data loading (replace with your dataset)
dfx = df.drop(['y'],axis=1)
dfy = df.drop(['x'],axis=1)
dfx1 , dfx2 = dfx.drop(['x'], axis=1), dfx['x']
dfy1 , dfy2 = dfy.drop(['y'], axis=1), dfy['y']


# Split the dataset into training and testing sets
X_train1, X_test1, y_train1, y_test1 = train_test_split(dfx1, dfx2, test_size=0.25, random_state=42)
X_train2, X_test2, y_train2, y_test2 = train_test_split(dfy1, dfy2, test_size=0.25, random_state=42)



# Initialize the Extra Trees Regressor
# Assuming you have a multi-output regression problem, ExtraTreesRegressor supports this by default.
# You can adjust n_estimators and other parameters as needed.
etrX = ExtraTreesRegressor(n_estimators=900, max_depth=None, random_state=42)
etrY = ExtraTreesRegressor(n_estimators=900, max_depth=None, random_state=42)


# Train the model
etrX.fit(X_train1, y_train1)
etrY.fit(X_train2, y_train2)

# Predict on the test set
prdX = etrX.predict(X_test1)
prdY = etrY.predict(X_test2)

# Evaluate the model
mseX = mean_squared_error(y_test1, prdX)
print(f'Mean Squared Error (X): {mseX}')

mseY = mean_squared_error(y_test2, prdY)
print(f'Mean Squared Error (Y): {mseY}')

# print(f'Eucld:{euclidean_distance_loss(y_test1,prdX)}')
# print(f'Eucld:{euclidean_distance_loss(y_test2,prdY)}')
# If you have multiple outputs, mse will return an array of errors for each output.


Mean Squared Error (X): 10.115061535556707
Mean Squared Error (Y): 14.47686018984684


In [4]:
df_test = pd.read_csv('/kaggle/input/evaluation/evaluation.csv')


In [5]:
ix = 0,7,12,15,16,17
noise = [f"rms[{i}]" for i in range(18)]
noise += [f"pmax[{i}]" for i in ix]
noise += [f"negpmax[{i}]" for i in ix]
noise += [f"tmax[{i}]" for i in ix]
noise += [f"area[{i}]" for i in ix]
df_test = df_test.drop(columns = noise)

In [6]:
ids = df_test['Id']
features_df = df_test.drop(columns=['Id'])
predX = etrX.predict(features_df)
predY = etrY.predict(features_df)
predictions_flattenedX = predX.flatten()
predictions_flattenedY = predY.flatten()
combined_predictions = [f"{x}|{y}" for x, y in zip(predictions_flattenedX, predictions_flattenedY)]
result_df = pd.DataFrame({'Id': ids, 'Predicted': combined_predictions})
result_df.to_csv('evalsu.csv', index=False)